### Persiapaan 

In [2]:
import pandas as pd

In [3]:
dataset=pd.read_csv('lda_content.csv')
dataset.head(5)

,processed_content
0,kumaha barudak well yuk ramaikan pesta anak no...
1,kustini sri purnomo memiliki strategi jitu men...
2,peraih emas olimpiade tokyo greysia polii bica...
3,kapolri jenderal lisyto sigit prabowo bicara s...
4,menteri bumn erick thohir bicara bangsa indone...
